In [2]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


In [ ]:
data = pd.read_excel('/content/drive/My Drive/Master thesis/DataSet-forUsing/4-hate_speech_mlma_ar_dataset.xlsx')
print("Sample from dataset",data.info())
# data = data[['tweet','sentiment']]



# data = data[data.tweet != '#NAME?']
# print("data_is_null \n",data.isnull().sum())

# print("Sample from dataset [Tweets,Sentiment]",data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353 entries, 0 to 3352
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           3353 non-null   int64 
 1   tweet                3345 non-null   object
 2   sentiment            3353 non-null   object
 3   directness           3353 non-null   object
 4   annotator_sentiment  3353 non-null   object
 5   target               3353 non-null   object
 6   group                3353 non-null   object
dtypes: int64(1), object(6)
memory usage: 183.5+ KB
Sample from dataset None


In [ ]:
# Drop rows that contain empty tweet
print("data_is_null \n",data.isnull().sum())
data=data[data.tweet.str.contains('#NAME?',na=False)]
data.info()

data_is_null 
 Unnamed: 0             0
tweet                  8
sentiment              0
directness             0
annotator_sentiment    0
target                 0
group                  0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           0 non-null      int64 
 1   tweet                0 non-null      object
 2   sentiment            0 non-null      object
 3   directness           0 non-null      object
 4   annotator_sentiment  0 non-null      object
 5   target               0 non-null      object
 6   group                0 non-null      object
dtypes: int64(1), object(6)
memory usage: 0.0+ bytes


In [ ]:
data = pd.read_excel('/content/drive/My Drive/Master thesis/DataSet-forUsing/4-hate_speech_mlma_ar_dataset.xlsx',usecols=['tweet', 'sentiment'])

data["HS_NOT"] = data.apply(lambda x: "Not-HS" if x["sentiment"] in ['normal'] else "HS",axis =1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353 entries, 0 to 3352
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      3345 non-null   object
 1   sentiment  3353 non-null   object
 2   HS_NOT     3353 non-null   object
dtypes: object(3)
memory usage: 78.7+ KB


In [ ]:
texts = pd.concat([data['tweet']],axis=0).astype("str")   
# from keras.preprocessing.text import Tokenize
from keras.preprocessing.text import Tokenizer
print(texts)
tokeniser = Tokenizer(num_words=2000,split=' ')
tokeniser.fit_on_texts(texts)
X = tokeniser.texts_to_sequences(texts)



0       صلاة الفجر خير لك من ترديد بول البعير وسبي الن...
1       صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...
2       طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...
3       @user @user انا اوافقك بخصوص السوريين و العراق...
4       هذه السعودية التي شعبها شعب الخيم و بول البعير...
                              ...                        
3348    @user يمشي بخطا ادارتها قيد من الأمريكان ونهب ...
3349    @user مهما حصل هندوس عليهم شويه الرويبضه بس نخ...
3350             الكلب لا يعض اذن اخوه كذابين خنازير @url
3351    @user لأنه صغير ويكتب في قناة خنازير فلن نقرأ ...
3352        الحريم أجمع افلوس وايفون قديم وواتساب ?? ساعة
Name: tweet, Length: 3353, dtype: object


In [ ]:
# pad: to make all input of same length
X = pad_sequences(X)
print(X[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
  1958 1324 1325   90    3    4    5 1959  236 1960 1961]
 [   0    0    0    0    0    0    0    0    0    0    0  993  171  290
   616 1326   24  617 1962 1327  141  754  324  755    2]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  172   64   30    3  215    2]
 [   0    0    0    0    0    0    0    0    0    0    1    1   34  756
   757   10  618  142  756  325   65 1963   61 1328    2]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0  105   97  101   86   10    4    5   96  158  102]]


In [ ]:
embed_dim = 128
lstm_out = 196
# import time

# model = Sequential()
# model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
# model.add(
# model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(2,activation='softmax'))
# start=time.time()
# model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print("Time to compile model:",time.time()-start)
# print(model.summary())

####
# The LSTM architecture
model=Sequential()
model.add(Embedding(15000,512,mask_zero=True))
model.add(LSTM(512,dropout=0.1, recurrent_dropout=0.1,return_sequences=True))
model.add(LSTM(256,dropout=0.1, recurrent_dropout=0.1,return_sequences=False))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# Compiling the RNN

# Fitting to the training set
# regressor.fit(X_train,Y_train,epochs=10,batch_size=32, verbose = 2)
# summary
print(model.summary())

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, None, 512)         7680000   
_________________________________________________________________
lstm_64 (LSTM)               (None, None, 512)         2099200   
_________________________________________________________________
lstm_65 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_28 (Dense)             (None, 2)                 514       
Total params: 10,567,170
Trainable params: 10,567,170
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
print((data['HS_NOT']).values)
Y = pd.get_dummies(data['HS_NOT']).values
print(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

['HS' 'HS' 'HS' ... 'HS' 'HS' 'Not-HS']
[[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [0 1]]
(2850, 25) (2850, 2)
(503, 25) (503, 2)


In [ ]:
model.fit(X_train, Y_train, epochs=4, batch_size=100, verbose=2)


Epoch 1/4
29/29 - 44s - loss: 0.5568 - accuracy: 0.7235
Epoch 2/4
29/29 - 44s - loss: 0.3976 - accuracy: 0.8161
Epoch 3/4
29/29 - 44s - loss: 0.2942 - accuracy: 0.8761
Epoch 4/4
29/29 - 44s - loss: 0.2220 - accuracy: 0.9042


In [ ]:
twt = ['انا اوافقك بخصوص السوريين و العراقيين اما بخصوص السعودي مو بحاجه انه يقعد بالكويت بدو']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokeniser.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("hate")
elif (np.argmax(sentiment) == 1):
    print("normal")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0   34  756  757   10  618  142  756  325   65 1963   61 1328 1951]]
1/1 - 0s
normal
